https://neuraldatascience.io/8-mri/nifti.html

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import nibabel as nib
import os
import nilearn as nil


In [ ]:
%pip install psutil gputil
%pip install pywidgets - -user


## Image Data

In [ ]:
brain_vol = nib.load(
    '..\\eda\\download\\ADNI\\002_S_0413\\MPR____N3__Scaled\\2006-05-19_16_17_47.0\\I40657\\ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070216232854688_S14782_I40657.nii')


In [ ]:
display(brain_vol)
print(brain_vol.header)


In [ ]:
brain_vol_data = brain_vol.get_fdata()
type(brain_vol_data)


In [ ]:
brain_vol_data.shape


In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Load MRI data
brain_vol = nib.load(
    '..\\eda\\download\\ADNI\\002_S_0413\\MPR____N3__Scaled\\2006-05-19_16_17_47.0\\I40657\\ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070216232854688_S14782_I40657.nii')
data = brain_vol.get_fdata()

# Function to visualize a slice


def visualize_slice(slice_idx):
    plt.imshow(data[slice_idx, :, :], cmap="gray", origin="lower")
    plt.title(f'Slice {slice_idx}')
    plt.axis('off')
    plt.show()


# Create an interactive slider
interact(visualize_slice, slice_idx=widgets.IntSlider(
    min=0, max=data.shape[0]-1, step=1, value=data.shape[0]//2))


In [ ]:
%pip install psutil gputil


In [ ]:
import plotly.graph_objects as go
import time
import numpy as np
import threading
import GPUtil
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt


vol = brain_vol_data
volume = vol.T
r, c = volume[0].shape
print(r, c)

# Define frames
nb_frames = 100

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
),
    name=str(k)  # you need to name the frame for the animation to behave properly
)
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
))

# Your existing code to load 'brain_vol_data' goes here...

vol = brain_vol_data
volume = vol.T
r, c = volume[0].shape

# Your existing code to define frames and layout goes here...

# Function to perform the visualization


def visualize():
    fig.show()

# Function to monitor GPU usage


def monitor_gpu(duration):
    gpu_loads = []
    for _ in range(duration):
        gpus = GPUtil.getGPUs()
        gpu_load = gpus[0].load if gpus else 0
        gpu_loads.append(gpu_load)
        print(f"GPU Load: {gpu_load*100:.2f}%")
        time.sleep(1)
    print("Average GPU Load:", np.mean(gpu_loads) * 100)


# Duration to monitor usage (in seconds)
monitor_duration = 60  # e.g., 60 seconds

# Start the visualization in a separate thread
viz_thread = threading.Thread(target=visualize)
viz_thread.start()

# Monitor GPU usage
monitor_gpu(monitor_duration)

# Wait for the visualization to finish if it's still running
viz_thread.join()
